# RL Trading Agent - Getting Started

This notebook demonstrates how to use the RL trading agent framework.


## 1. Setup and Data Loading

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from data.data_loader import DataLoader
from environment.trading_env import TradingEnvironment
from agents.dqn_agent import DQNAgent
from visualization.plotting import TradingVisualizer

In [ ]:
# Load and prepare data
data_loader = DataLoader('../config/config.yaml')
train_data, test_data = data_loader.prepare_data('AAPL')

print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print(f"\nFeatures: {list(train_data.columns)}")

## 2. Environment Setup

In [ ]:
# Create trading environment
env = TradingEnvironment(train_data, '../config/config.yaml')

print(f"Observation space: {env.observation_space.shape}")
print(f"Action space: {env.action_space.n}")
print(f"Initial balance: ${env.initial_balance:,}")

## 3. Agent Initialization

In [ ]:
# Initialize DQN agent
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size, '../config/config.yaml')

print(f"Agent initialized with state size: {state_size}")
print(f"Action size: {action_size}")
print(f"Device: {agent.device}")

## 4. Quick Training Demo

In [ ]:
# Quick training demo (10 episodes)
episodes = 10
rewards = []

for episode in range(episodes):
    state, _ = env.reset()
    total_reward = 0
    
    while True:
        action = agent.act(state, training=True)
        next_state, reward, done, truncated, info = env.step(action)
        
        agent.remember(state, action, reward, next_state, done)
        
        if len(agent.memory) > agent.batch_size:
            agent.replay()
        
        state = next_state
        total_reward += reward
        
        if done or truncated:
            break
    
    rewards.append(total_reward)
    print(f"Episode {episode + 1}: Reward = {total_reward:.2f}, Portfolio = ${info['portfolio_value']:.0f}")

plt.figure(figsize=(10, 4))
plt.plot(rewards)
plt.title('Training Rewards (Demo)')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.grid(True)
plt.show()

## 5. Evaluation

In [ ]:
# Evaluate the agent
test_env = TradingEnvironment(test_data, '../config/config.yaml')
state, _ = test_env.reset()
portfolio_values = [test_env.initial_balance]

while True:
    action = agent.act(state, training=False)  # No exploration
    next_state, reward, done, truncated, info = test_env.step(action)
    
    portfolio_values.append(info['portfolio_value'])
    state = next_state
    
    if done or truncated:
        break

# Plot results
visualizer = TradingVisualizer()
visualizer.plot_portfolio_performance(portfolio_values, test_env.trade_history)

# Print final stats
stats = test_env.get_portfolio_stats()
print(f"\nFinal Portfolio Value: ${stats['final_portfolio_value']:,.2f}")
print(f"Total Return: {stats['total_return']:.2f}%")
print(f"Total Trades: {stats['total_trades']}")
print(f"Win Rate: {stats['win_rate']:.1f}%")

## 6. Next Steps

- Run the full training script: `python ../train_agent.py`
- Evaluate trained models: `python ../evaluate_agent.py`
- Experiment with different hyperparameters in `config/config.yaml`
- Try different stocks by modifying the symbols list
- Implement additional technical indicators